# Módulo 5 Tarefa 1


## Base de nascidos vivos do DataSUS


### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

# Esta analise irá depurar o documento do DataSUS [SINAS 2019](https://datasus.saude.gov.br/)


## Este arquivo de dados tem muitas variáveis, estou descrevendo as mais importantes:



| Header  | Definition | Describe|
|---------|------------|---------|
|APGAR5|Apgar no quinto minuto|00 a 10 | 10
|GESTACAO|Semanas de gestação |9: Ignorado |
|||Menos de 22 semanas |
|||22 a 27 semanas  |
|||28 a 31 semanas |
|||32 a 36 semanas |
|||32 a 36 semanas |
|||37 a 41 semanas |
|||42 semanas e mais |
|GRAVIDEZ|Tipo de gravidez|9: Ignorado |
|||1: Única|
|||2: Dupla|
|||3: Tripla e mais|

## Amostragem inicial do DataFrame:

In [1]:
import pandas as pd
import requests
import re

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')

#Não há duplicados
print(sinasc.shape)
sinasc.drop_duplicates().shape 

#sinasc.head()

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui

#porcentagem de NaN na amostragem
sinasc.isna().sum() / len(sinasc) * 100

ORIGEM        0.000000
CODESTAB      0.425485
CODMUNNASC    0.000000
LOCNASC       0.000000
IDADEMAE      0.000000
                ...   
munResUf      0.000000
munResLat     0.003700
munResLon     0.003700
munResAlt     0.003700
munResArea    0.003700
Length: 69, dtype: float64

Recorte das colunas mais importantes para a analise: 

In [3]:
# 3) seu código aqui

#recortanto as colunas de interesse desse estudo e verificando a % de NaN
colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc_df = sinasc[colunas]

#fazendo a contagem de NaN do DF
 
sinasc_df.isna().sum() / len(sinasc_df) * 100



LOCNASC       0.000000
IDADEMAE      0.000000
ESTCIVMAE     1.172858
ESCMAE        1.154358
QTDFILVIVO    5.819890
GESTACAO      4.558236
GRAVIDEZ      0.292289
CONSULTAS     0.000000
APGAR5        0.381086
dtype: float64

## Tratamento dos dados da variável resposta 'APGAR5'

In [4]:
# 4) seu código aqui

#Código que remove as linhas NaN da variável resposta
sinasc_df.dropna(subset=['APGAR5'], inplace=True)

#fazendo a contagem novamente, vemos que a % de NaN na coluna é zero 
sinasc_df.isna().sum() / len(sinasc_df) * 100



C:\Users\djang\AppData\Local\Temp\ipykernel_12176\1844497661.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.dropna(subset=['APGAR5'], inplace=True)


LOCNASC       0.000000
IDADEMAE      0.000000
ESTCIVMAE     1.169916
ESCMAE        1.151346
QTDFILVIVO    5.816156
GESTACAO      4.516249
GRAVIDEZ      0.282266
CONSULTAS     0.000000
APGAR5        0.000000
dtype: float64

## Tratamento de NaN das variáveis explicativas 

### Tratando a coluna explicativa 'QTDFILVIVO'

Padronizei a variável 'QTDfilvivo' como 0 (sem filho) 

In [5]:
# 6) Seu código aqui

#tratando a coluna 'QTDfilvivo', padronizando NaN como 0 
sinasc['QTDFILVIVO'].fillna(0, inplace = True)
sinasc_df['QTDFILVIVO'].unique()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\3263153164.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc['QTDFILVIVO'].fillna(0, inplace = True)


array([ 0.,  1.,  2.,  3., nan,  4.,  6.,  5.,  7., 12.,  9., 11.,  8.,
       30., 10., 14.])

### Tratando coluna explicativa 'CONSULTAS' e 'ESTCIVMAE'

In [6]:
# 5) seu código aqui
#Código que identifica NaN na coluna  
sinasc_df[['CONSULTAS', 'ESTCIVMAE']].isna().sum() / len(sinasc_df) * 100

#Código que altera o NaN para 9 (ignorado).
sinasc_df['ESTCIVMAE'].fillna(9, inplace = True)
sinasc_df[['ESTCIVMAE']].value_counts()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\3899372781.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_df['ESTCIVMAE'].fillna(9, inplace = True)
C:\Users\djang\AppData\Local\Temp\ipykernel_12176\3899372781.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df['ESTCIVMAE'].fillna(9, inplace = True)


ESTCIVMAE
2.0          9906
1.0          8907
5.0          7145
9.0           459
4.0           451
3.0            57
Name: count, dtype: int64

### Tratando coluna explicativa 'ESCMAE'

A variável 'ESCMAE' tem 310 dados sem preenchimento (NaN), cerca de 1%. 


In [7]:
#Código que identifica NaN na coluna  
sinasc_df['ESCMAE'].isna().sum() / len(sinasc_df) * 100
#coluna 'ESCMAE' - padronizando os NaN como 9 (ignorado) 
sinasc_df['ESCMAE'].fillna(9, inplace = True)
sinasc_df['ESCMAE'].unique()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\4086122762.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_df['ESCMAE'].fillna(9, inplace = True)
C:\Users\djang\AppData\Local\Temp\ipykernel_12176\4086122762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df['ESCMAE'].fillna(9, inplace = True)


array(['8 a 11 anos', '12 anos ou mais', '1 a 3 anos', '4 a 7 anos', 9,
       'Nenhuma'], dtype=object)

### Tratando coluna explicativa 'ESTCIVMAE'

A variável 'ESTCIVMAE' tem 315 dados sem preenchimento (NaN), cerca de 1%, por tanto irei remover os dados pois não são relevantes para a analise. 

In [8]:
#Código que identifica NaN na coluna  
sinasc_df['ESTCIVMAE'].isna().sum() / len(sinasc_df) * 100
#Código que remove a linha NaN
sinasc_df.dropna(subset = ['ESTCIVMAE'], inplace = True)
sinasc_df['ESTCIVMAE'].isna().sum()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\3630356161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.dropna(subset = ['ESTCIVMAE'], inplace = True)


0

In [9]:
# 7) seu código aqui

### Tratando coluna explicativa 'GESTACAO'

A variável 'GESTACAO' tem 1216 dados sem preenchimento (NaN), cerca de 4.5%. 

In [10]:
#Código que identifica NaN na coluna  
sinasc_df['GESTACAO'].isna().sum() / len(sinasc_df) * 100
sinasc_df['GESTACAO'].value_counts()
sinasc_df['GESTACAO'].unique()

array(['37 a 41 semanas', '42 semanas e mais', '32 a 36 semanas',
       '22 a 27 semanas', '28 a 31 semanas', nan, 'Menos de 22 semanas'],
      dtype=object)

In [11]:
#criando um DataFrame com as semanas e quantidades por categoria 
data = {
    'Gestação': ['37 a 41 semanas', '32 a 36 semanas', '42 semanas e mais', 
                 '28 a 31 semanas', '22 a 27 semanas', 'Menos de 22 semanas'],
    'Quantidade': [22523, 2234, 698, 162, 85, 7]
}
sinasc_df_gestacao = pd.DataFrame(data)

#Mapeamento das categorias para valores médios de semana (média ponderada das semanas)
mapa_gestacao = {
    '37 a 41 semanas': 39,
    '32 a 36 semanas': 34,
    '42 semanas e mais': 42,
    '28 a 31 semanas': 29.5,
    '22 a 27 semanas': 24.5,
    'Menos de 22 semanas': 21
}

#criando uma coluna nova com os valores no DataFrame raiz 
sinasc_df['GESTACAOVALORES'] = sinasc_df['GESTACAO'].map(mapa_gestacao)

#calculando a média ponderada, considerando a quantidade de cada grupo

media_ponderada = (sinasc_df['GESTACAOVALORES'] * sinasc_df_gestacao['Quantidade']).sum() / sinasc_df_gestacao['Quantidade'].sum()
media_ponderada

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\2909767642.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df['GESTACAOVALORES'] = sinasc_df['GESTACAO'].map(mapa_gestacao)


39.0

A média ponderada entre essas semanas é 39, se encaixando entre 37 e 41 anos, por tanto, irei preencher os NaN nessa categoria.

In [12]:
#código que muda os NaA para o grupo que se encaixa na média 
sinasc_df['GESTACAO'].fillna('37 a 41 semanas', inplace = True)
sinasc_df['GESTACAO'].unique()
sinasc_df['GESTACAO'].isna().sum()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\257020812.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sinasc_df['GESTACAO'].fillna('37 a 41 semanas', inplace = True)
C:\Users\djang\AppData\Local\Temp\ipykernel_12176\257020812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df['GESTACAO'].fillna('37 a 41 semanas', inplace =

0

### Tratando coluna 'GRAVIDEZ'

A variável 'GRAVIDEZ' tem 79 dados sem preenchimento (NaN), cerca de 0.30%, por tanto irei remover os dados pois não são relevantes para a analise. 

In [13]:
#Código que identifica NaN na coluna  
sinasc_df['GRAVIDEZ'].isna().sum() / len(sinasc_df) * 100
sinasc_df['GRAVIDEZ'].value_counts()

#Código que remove a linha NaN
sinasc_df.dropna(subset = ['GRAVIDEZ'], inplace = True)
sinasc_df['GRAVIDEZ'].isna().sum()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\970446619.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.dropna(subset = ['GRAVIDEZ'], inplace = True)


0

### Tratando coluna 'QTDFILVIVO'

A variável 'QTDFILVIVO' tem 1534 dados sem preenchimento (NaN), cerca de 5.76%, por tanto irei remover os dados pois não são relevantes para a analise. 

In [23]:
#Código que identifica NaN na coluna  
sinasc_df['QTDFILVIVO'].isna().sum() / len(sinasc_df) * 100

#Código que remove a linha NaN
sinasc_df.dropna(subset = ['QTDFILVIVO'], inplace = True)
sinasc_df['QTDFILVIVO'].isna().sum()

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\4201612205.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.dropna(subset = ['QTDFILVIVO'], inplace = True)


0

Abaixo podemos ver que todas as linhas estão tratadas.

In [14]:
sinasc_df.head()
sinasc_df.isna().sum() / len(sinasc_df) * 100 

LOCNASC            0.000000
IDADEMAE           0.000000
ESTCIVMAE          0.000000
ESCMAE             0.000000
QTDFILVIVO         5.795374
GESTACAO           0.000000
GRAVIDEZ           0.000000
CONSULTAS          0.000000
APGAR5             0.000000
GESTACAOVALORES    4.469440
dtype: float64

## Categorizando a variavél resposta

Irei categorizar os dados do 'APGAR5' para conceguir medir a frequencia.

In [15]:
# 8) seu código aqui
#Código que localiza a linha com o condicional passado e adiciona em uma nova coluna com o nome 'APGAR5_FAIXA'
sinasc_df.loc[sinasc_df['APGAR5'] >= 8, 'APGAR5FAIXA'] = 'Normal'
sinasc_df.loc[(sinasc_df['APGAR5'] <= 6) & (sinasc_df['APGAR5'] <= 7), 'APGAR5FAIXA'] = 'Asfixia leve'
sinasc_df.loc[(sinasc_df['APGAR5'] <= 4) & (sinasc_df['APGAR5'] <= 5), 'APGAR5FAIXA'] = 'Asfixia moderada'
sinasc_df.loc[(sinasc_df['APGAR5'] == 0) & (sinasc_df['APGAR5'] <= 3), 'APGAR5FAIXA'] = 'Asfixia severa'

C:\Users\djang\AppData\Local\Temp\ipykernel_12176\237294344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_df.loc[sinasc_df['APGAR5'] >= 8, 'APGAR5FAIXA'] = 'Normal'


Agora conseguimos tirar a frequencia da coluna 'APGAR5' baseada na categorização abaixo: 

|Categorias|Faixa|
|----------|-----|
|8 e 10| Normal|
|6 e 7 | Asfixia leve|
|4 e 5 | Asfixia moderada|
|0 e 3 | Asfixia severa|'.|

In [16]:
#Código que faz a contagem de cada faixa. 
sinasc_df['APGAR5FAIXA'].value_counts()

APGAR5FAIXA
Normal              26388
Asfixia leve          124
Asfixia moderada       80
Asfixia severa         17
Name: count, dtype: int64

Inicialmente já podemos ver que uma porcentagem muito pequena de crianças, nascem com um nivel anormal de asfixia, podemos trabalhar em cima dessas para enteder mais os motivos que podem levar a isso. 

In [17]:
sinasc_df.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,GESTACAOVALORES,APGAR5FAIXA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,39.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,39.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,39.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,39.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,39.0,Normal


In [24]:
# 9) seu código aqui

# Lista original de colunas
#colunas = ['LOC_NASC', 'IDADE_MAE', 'EST_CIV_MAE', 'ESC_MAE', 'QTD_FIL_VIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR_5', 'GESTACAO_VALORES', 'APGAR_5_FAIXA']

# Exibir as colunas modificadas
#colunas_modificadas = [col.lower() for col in colunas_novas]

#sinasc_df.columns = colunas_modificadas
#sinasc_df.head()

#NaN tratados 
sinasc_df.isna().sum()

LOCNASC               0
IDADEMAE              0
ESTCIVMAE             0
ESCMAE                0
QTDFILVIVO            0
GESTACAO              0
GRAVIDEZ              0
CONSULTAS             0
APGAR5                0
GESTACAOVALORES    1082
APGAR5FAIXA         231
dtype: int64